In [8]:
import gym
env = gym.make("Taxi-v3",render_mode='ansi')
env.reset()
print(env.render())

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

state = env.encode(0, 2, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
print('type: ', type(state))
print("decode:", list(env.decode(state)))
env.d = state
print(env.render())
env.P[state]

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)
State: 48
type:  <class 'int'>
decode: [0, 2, 2, 0]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




{0: [(1.0, 148, -1, False)],
 1: [(1.0, 48, -1, False)],
 2: [(1.0, 68, -1, False)],
 3: [(1.0, 48, -1, False)],
 4: [(1.0, 48, -10, False)],
 5: [(1.0, 48, -10, False)]}

In [11]:
epochs = 0
penalties, reward = 0, 0

frames = [] # for animation
done = False

while not done:
    action = env.action_space.sample()
    
    state, reward, terminated, truncated, info = env.step(action) 
    done = terminated or truncated
    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward
            }
        )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1
Penalties incurred: 0


In [14]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print("Decode:", list(env.decode(frame['state'])))
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 1
State: 74
Decode: [0, 3, 3, 2]
Action: 1
Reward: -1


In [15]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])


In [16]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    env.reset()
    state = env.d

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, terminated, truncated, info = env.step(action) 
        done = terminated or truncated
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    all_epochs.append(epochs)
    all_penalties.append(penalties)
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100
Episode: 200
Episode: 300
Episode: 400
Episode: 500
Episode: 600
Episode: 700
Episode: 800
Episode: 900
Episode: 1000
Episode: 1100
Episode: 1200
Episode: 1300
Episode: 1400
Episode: 1500
Episode: 1600
Episode: 1700
Episode: 1800
Episode: 1900
Episode: 2000
Episode: 2100
Episode: 2200
Episode: 2300
Episode: 2400
Episode: 2500
Episode: 2600
Episode: 2700
Episode: 2800
Episode: 2900
Episode: 3000
Episode: 3100
Episode: 3200
Episode: 3300
Episode: 3400
Episode: 3500
Episode: 3600
Episode: 3700
Episode: 3800
Episode: 3900
Episode: 4000
Episode: 4100
Episode: 4200
Episode: 4300
Episode: 4400
Episode: 4500
Episode: 4600
Episode: 4700
Episode: 4800
Episode: 4900
Episode: 5000
Episode: 5100
Episode: 5200
Episode: 5300
Episode: 5400
Episode: 5500
Episode: 5600
Episode: 5700
Episode: 5800
Episode: 5900
Episode: 6000
Episode: 6100
Episode: 6200
Episode: 6300
Episode: 6400
Episode: 6500
Episode: 6600
Episode: 6700
Episode: 6800
Episode: 6900
Episode: 7000
Episode: 7100
Episode: 7200
E

In [7]:
# import pandas as pd
# data = {'epochs':all_epochs,'penalties':all_penalties}
# frame = pd.DataFrame(data)
# frame.to_excel('taxi_output.xlsx')


In [17]:
print(list(q_table))
# np.savetxt('q_table_taxi.csv', q_table, delimiter=',')

[array([0., 0., 0., 0., 0., 0.]), array([ -2.41837057,  -2.36395098,  -2.41837058,  -2.36395109,
        -2.27325184, -11.36395083]), array([ -1.87014399,  -1.45024007,  -1.87014396,  -1.45024002,
        -0.7504    , -10.45023932]), array([ -2.36395101,  -2.27325184,  -2.3639511 ,  -2.27325184,
        -2.1220864 , -11.27325143]), array([-2.4961915 , -2.49694754, -2.4961915 , -2.49658355, -8.62026403,
       -7.326364  ]), array([0., 0., 0., 0., 0., 0.]), array([-2.4961915 , -2.49704909, -2.4961915 , -2.49687946, -9.01125112,
       -8.11715907]), array([-2.48236806, -2.48385905, -2.48236806, -2.485413  , -9.4636933 ,
       -9.25165359]), array([-2.27325184, -2.34030368, -2.37976785, -2.34667762, -9.39666004,
       -8.86016568]), array([-2.47061344, -2.47456395, -2.48033813, -2.4786809 , -8.47858057,
       -9.27821232]), array([0., 0., 0., 0., 0., 0.]), array([-2.4510224 , -2.46595955, -2.46968178, -2.45828612, -9.26582019,
       -9.84400657]), array([ -2.48942084,  -2.4919416 ,  

In [22]:
from IPython.display import clear_output
from time import sleep
total_epochs, total_penalties = 0, 0
episodes = 1

for _ in range(episodes):
    env.reset()
    state=env.s
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        clear_output(wait=True)
        action = np.argmax(q_table[state])
        state, reward, terminated, truncated, info = env.step(action) 
        print(env.render())
        print(list(env.decode(state)))
        sleep(0.2)
        done = terminated or truncated

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

[0, 0, 0, 0]
Results after 1 episodes:
Average timesteps per episode: 13.0
Average penalties per episode: 0.0
